In [17]:
import os
import json

In [18]:
# load json file and read line by line
def load_json_file(file_path):
    data = []
    with open(file_path, 'r') as f:
        for line in f:
            data.append(json.loads(line))
    return data

In [19]:
data_part = "all"
train_path = f"../data/processed/train_AST_abcd_waction_flow_{data_part}.json"

train_data = load_json_file(train_path)

In [20]:
print(train_data[0]['target'])
print(train_data[1]['target'])

pull-up-account [albert sanders]
validate-purchase [<username>, <username>, <username>]


#### process the actions described in the format of natural language to the structured format

In [21]:
mapping_dict = {
  "pull-up-account": "pull up the costumer account",
  "enter-details": "enter details",
  "verify-identity": "verify costumer identity",
  "make-password": "create new password",
  "search-timing": "search timing",
  "search-policy": "check policy",
  "validate-purchase": "validate purchase",
  "search-faq": "search faq",
  "membership": "check membership level",
  "search-boots": "search for boots",
  "try-again": "ask costumer to try again",
  "ask-the-oracle": "ask oracle",
  "update-order": "update order information",
  "promo-code": "offer promo code",
  "update-account": "update costumer account",
  "search-membership": "get memberships information",
  "make-purchase": "make purchase",
  "offer-refund": "offer refund",
  "notify-team": "notify team",
  "record-reason": "record reason",
  "search-jeans": "search for jeans",
  "shipping-status": "get shipping status",
  "search-shirt": "search for shirt",
  "instructions": "check instructions",
  "search-jacket": "search for jacket",
  "log-out-in": "ask costumer log out log in",
  "select-faq": "select topic in faq",
  "subscription-status": "get subscription status",
  "send-link": "send link to costumer",
  "search-pricing": "check pricing"
}

# conver key to value and value to key
def convert_dict(mapping_dict):
    new_dict = {}
    for key, value in mapping_dict.items():
        new_dict[value] = key
    return new_dict

new_mapping_dict = convert_dict(mapping_dict)
print(new_mapping_dict)

{'pull up the costumer account': 'pull-up-account', 'enter details': 'enter-details', 'verify costumer identity': 'verify-identity', 'create new password': 'make-password', 'search timing': 'search-timing', 'check policy': 'search-policy', 'validate purchase': 'validate-purchase', 'search faq': 'search-faq', 'check membership level': 'membership', 'search for boots': 'search-boots', 'ask costumer to try again': 'try-again', 'ask oracle': 'ask-the-oracle', 'update order information': 'update-order', 'offer promo code': 'promo-code', 'update costumer account': 'update-account', 'get memberships information': 'search-membership', 'make purchase': 'make-purchase', 'offer refund': 'offer-refund', 'notify team': 'notify-team', 'record reason': 'record-reason', 'search for jeans': 'search-jeans', 'get shipping status': 'shipping-status', 'search for shirt': 'search-shirt', 'check instructions': 'instructions', 'search for jacket': 'search-jacket', 'ask costumer log out log in': 'log-out-in', 

In [22]:
all_transitions = []
tmp_all_transitions = {}
# for i in range(len(train_data)):
#     # if i == 10:
#     #     break
#     # print(new_mapping_dict[train_data[i]['target']])
#     target_pieces = train_data[i]['target'].split(';')
#     # pull up the costumer account [alessandro phoenix]
#     # only get the first part before the first '['
#     for j in range(len(target_pieces)):
#         target_pieces[j] = target_pieces[j].split('[')[0].strip()
    
#     for j in range(len(target_pieces)):
#         if target_pieces[j] in new_mapping_dict:
#             target_pieces[j] = new_mapping_dict[target_pieces[j]]
#         else:
#             print("not found")

#     # print(target_pieces)

#     all_transitions.append(target_pieces)

for i in range(len(train_data)):
    if train_data[i]['flow'] == "storewide_query":
        action = train_data[i]['target'].split(' ')[0].strip()
        if train_data[i]['convo_id'] not in tmp_all_transitions:
            tmp_all_transitions[train_data[i]['convo_id']] = []
        tmp_all_transitions[train_data[i]['convo_id']].append(action)

for key, value in tmp_all_transitions.items():
    all_transitions.append(value)

print("all_transitions: ", all_transitions)

# count the freequency of the first action in each dialogue
action_count = {}
for i in range(len(all_transitions)):
    if all_transitions[i][0] not in action_count:
        action_count[all_transitions[i][0]] = 1
    else:
        action_count[all_transitions[i][0]] += 1

print("action_count: ", json.dumps(action_count, indent=2))

# convert to rate
action_rate = {}
total_dialogue = len(all_transitions)
for key, value in action_count.items():
    action_rate[key] = value / total_dialogue

print("action_rate: ", json.dumps(action_rate, indent=2))

all_transitions:  [['search-faq', 'search-policy', 'select-faq'], ['search-faq', 'search-membership', 'search-policy', 'select-faq'], ['search-faq', 'search-timing', 'select-faq'], ['search-faq', 'search-membership'], ['pull-up-account', 'verify-identity', 'subscription-status', 'send-link', 'enter-details'], ['search-faq', 'search-timing', 'search-timing', 'search-membership'], ['search-faq', 'search-pricing', 'select-faq'], ['search-faq', 'search-pricing'], ['search-faq', 'search-membership', 'search-policy'], ['search-faq', 'search-membership', 'select-faq', 'select-faq'], ['search-faq', 'search-policy', 'select-faq'], ['search-faq', 'search-jeans', 'search-pricing', 'search-jeans', 'search-timing', 'search-pricing', 'select-faq'], ['pull-up-account', 'search-faq', 'search-timing', 'select-faq'], ['search-faq', 'search-membership'], ['search-faq', 'search-policy', 'search-membership', 'select-faq'], ['search-faq', 'search-membership'], ['search-faq', 'search-membership', 'select-faq

In [23]:
possible_states = mapping_dict.keys()
print(list(possible_states))
possible_states = list(possible_states)
# ['pull-up-account', 'enter-details', 'verify-identity', 'make-password', 'search-timing', 'search-policy', 'validate-purchase', 'search-faq', 'membership', 'search-boots', 'try-again', 'ask-the-oracle', 'update-order', 'promo-code', 'update-account', 'search-membership', 'make-purchase', 'offer-refund', 'notify-team', 'record-reason', 'search-jeans', 'shipping-status', 'search-shirt', 'instructions', 'search-jacket', 'log-out-in', 'select-faq', 'subscription-status', 'send-link', 'search-pricing']

['pull-up-account', 'enter-details', 'verify-identity', 'make-password', 'search-timing', 'search-policy', 'validate-purchase', 'search-faq', 'membership', 'search-boots', 'try-again', 'ask-the-oracle', 'update-order', 'promo-code', 'update-account', 'search-membership', 'make-purchase', 'offer-refund', 'notify-team', 'record-reason', 'search-jeans', 'shipping-status', 'search-shirt', 'instructions', 'search-jacket', 'log-out-in', 'select-faq', 'subscription-status', 'send-link', 'search-pricing']


In [24]:
# count the transition frequency between every two actions/state according to all_transitions
transition_count = {}
for i in range(len(all_transitions)):
    for j in range(len(all_transitions[i]) - 1):
        if (all_transitions[i][j], all_transitions[i][j+1]) in transition_count:
            transition_count[(all_transitions[i][j], all_transitions[i][j+1])] += 1
        else:
            transition_count[(all_transitions[i][j], all_transitions[i][j+1])] = 1

print("transition_count: ", transition_count)

# if no transition between two states, set the frequency to 0
for state1 in possible_states:
    for state2 in possible_states:
        if (state1, state2) not in transition_count:
            transition_count[(state1, state2)] = 0

print("transition_count: ", transition_count)

transition_count:  {('search-faq', 'search-policy'): 171, ('search-policy', 'select-faq'): 141, ('search-faq', 'search-membership'): 245, ('search-membership', 'search-policy'): 30, ('search-faq', 'search-timing'): 199, ('search-timing', 'select-faq'): 144, ('pull-up-account', 'verify-identity'): 19, ('verify-identity', 'subscription-status'): 8, ('subscription-status', 'send-link'): 6, ('send-link', 'enter-details'): 2, ('search-timing', 'search-timing'): 3, ('search-timing', 'search-membership'): 10, ('search-faq', 'search-pricing'): 188, ('search-pricing', 'select-faq'): 153, ('search-membership', 'select-faq'): 165, ('select-faq', 'select-faq'): 44, ('search-faq', 'search-jeans'): 9, ('search-jeans', 'search-pricing'): 7, ('search-pricing', 'search-jeans'): 2, ('search-jeans', 'search-timing'): 1, ('search-timing', 'search-pricing'): 8, ('pull-up-account', 'search-faq'): 46, ('search-policy', 'search-membership'): 13, ('select-faq', 'pull-up-account'): 6, ('pull-up-account', 'ask-t

In [25]:
# print the pairs number
counter = 0
for key, value in transition_count.items():
    counter += 1
    print(key, value)

print("counter: ", counter)

('search-faq', 'search-policy') 171
('search-policy', 'select-faq') 141
('search-faq', 'search-membership') 245
('search-membership', 'search-policy') 30
('search-faq', 'search-timing') 199
('search-timing', 'select-faq') 144
('pull-up-account', 'verify-identity') 19
('verify-identity', 'subscription-status') 8
('subscription-status', 'send-link') 6
('send-link', 'enter-details') 2
('search-timing', 'search-timing') 3
('search-timing', 'search-membership') 10
('search-faq', 'search-pricing') 188
('search-pricing', 'select-faq') 153
('search-membership', 'select-faq') 165
('select-faq', 'select-faq') 44
('search-faq', 'search-jeans') 9
('search-jeans', 'search-pricing') 7
('search-pricing', 'search-jeans') 2
('search-jeans', 'search-timing') 1
('search-timing', 'search-pricing') 8
('pull-up-account', 'search-faq') 46
('search-policy', 'search-membership') 13
('select-faq', 'pull-up-account') 6
('pull-up-account', 'ask-the-oracle') 13
('ask-the-oracle', 'promo-code') 6
('pull-up-account'

In [26]:
# count the frequency of each state
transition_matrix = {}
for state1 in possible_states:
    transition_matrix[state1] = {}
    for state2 in possible_states:
        transition_matrix[state1][state2] = 0
    
    for key, value in transition_count.items():
        if state1 == key[0]:
            transition_matrix[state1][key[1]] = value

print("transition_matrix: ", transition_matrix)
print(len(transition_matrix['pull-up-account']))

# convert count to probability
for state1 in possible_states:
    sum_count = sum(transition_matrix[state1].values())
    for state2 in possible_states:
        if sum_count != 0:
            transition_matrix[state1][state2] = transition_matrix[state1][state2] / sum_count
        
print("transition_matrix: ", transition_matrix)
print(len(transition_matrix['pull-up-account']))
print("sum of probability: ", sum(transition_matrix['pull-up-account'].values()))
print("sum of probability: ", sum(transition_matrix['enter-details'].values()))

transition_matrix:  {'pull-up-account': {'pull-up-account': 4, 'enter-details': 0, 'verify-identity': 19, 'make-password': 0, 'search-timing': 0, 'search-policy': 0, 'validate-purchase': 1, 'search-faq': 46, 'membership': 9, 'search-boots': 0, 'try-again': 0, 'ask-the-oracle': 13, 'update-order': 0, 'promo-code': 1, 'update-account': 0, 'search-membership': 0, 'make-purchase': 2, 'offer-refund': 0, 'notify-team': 0, 'record-reason': 2, 'search-jeans': 0, 'shipping-status': 1, 'search-shirt': 0, 'instructions': 0, 'search-jacket': 0, 'log-out-in': 0, 'select-faq': 0, 'subscription-status': 6, 'send-link': 0, 'search-pricing': 0}, 'enter-details': {'pull-up-account': 0, 'enter-details': 1, 'verify-identity': 0, 'make-password': 0, 'search-timing': 0, 'search-policy': 0, 'validate-purchase': 1, 'search-faq': 0, 'membership': 0, 'search-boots': 0, 'try-again': 0, 'ask-the-oracle': 0, 'update-order': 0, 'promo-code': 1, 'update-account': 3, 'search-membership': 0, 'make-purchase': 0, 'offer

In [27]:
# convert to mdp:
# here we use the dot format to represent the mdp
'''
digraph learned_mdp {
s0 [label="init"];
s1 [label="pull-up-account"];
s2 [label="end"];
s3 [label="enter-details"];
s0 -> s1  [label="pull-up-account:0.03125"];
s0 -> s3  [label="enter-details:0.03125"];
s0 -> s2  [label="end:0.03125"];
s1 -> s3  [label="pull-up-account:0.2202970297029703"];
s1 -> s1  [label="pull-up-account:0.7797029702970297"];
s1 -> s3  [label="enter-details:0.03125"];
s1 -> s2  [label="end:0.03125"];
s2 -> s1  [label="pull-up-account:0.03125"];
s2 -> s3  [label="enter-details:0.03125"];
s2 -> s2  [label="end:0.03125"];
s3 -> s1  [label="pull-up-account:0.03125"];
s3 -> s3  [label="enter-details:0.03125"];
s3 -> s2  [label="end:0.03125"];
__start0 [label="", shape=none];
__start0 -> s0  [label=""];
}
'''

# ['find_hotel', 'book_hotel', 'find_train', 'book_train', 'find_attraction', 'find_restaurant', 'book_restaurant', 'find_hospital', 'book_taxi', 'find_taxi', 'find_bus', 'find_police']

'''
action_rate:  {
  "pull-up-account": 0.6694050286283296,
  "search-faq": 0.2045058501369181,
  "enter-details": 0.006596962907642519,
  "offer-refund": 0.0011202389843166542,
  "verify-identity": 0.017301468757779437,
  "try-again": 0.035225292506845904,
  "validate-purchase": 0.0043564849390092105,
  "notify-team": 0.01157580283793876,
  "log-out-in": 0.032984814538212594,
  "record-reason": 0.003983071944236993,
  "make-purchase": 0.0008712969878018422,
  "instructions": 0.0028628329599203386,
  "shipping-status": 0.0017425939756036844,
  "membership": 0.0036096589494647746,
  "ask-the-oracle": 0.0008712969878018422,
  "subscription-status": 0.0013691809808314662,
  "update-account": 0.0004978839930296241,
  "make-password": 0.00024894199651481205,
  "update-order": 0.0003734129947722181,
  "promo-code": 0.0003734129947722181,
  "send-link": 0.00012447099825740602
}
'''

# convert transition_matrix to dot format
def convert_to_dot(transition_matrix):
    dot_str = "digraph learned_mdp {\n"
    dot_str += "s0 [label=\"init\"];\n"
    for i in range(len(possible_states)):
        dot_str += "s" + str(i+1) + " [label=\"" + possible_states[i] + "\"];\n"
    dot_str += "s31 [label=\"end\"];\n"    

    dot_str += "s0 -> s0  [label=\"init:1.0\"];\n"
    # dot_str += "s0 -> s1  [label=\"action:0.215\"];\n"
    # dot_str += "s0 -> s2  [label=\"action:0.00125\"];\n"
    # dot_str += "s0 -> s3  [label=\"action:0.22875\"];\n"
    # dot_str += "s0 -> s4  [label=\"action:0.0\"];\n"
    # dot_str += "s0 -> s5  [label=\"action:0.1675\"];\n"
    # dot_str += "s0 -> s6  [label=\"action:0.2875\"];\n"
    # dot_str += "s0 -> s7  [label=\"action:0.00125\"];\n"
    # dot_str += "s0 -> s8  [label=\"action:0.03625\"];\n"
    # dot_str += "s0 -> s9  [label=\"action:0.0\"];\n"
    # dot_str += "s0 -> s10  [label=\"action:0.0325\"];\n"
    # dot_str += "s0 -> s11  [label=\"action:0.0\"];\n"
    # dot_str += "s0 -> s12  [label=\"action:0.03\"];\n"
    for i in range(len(possible_states)):
        if possible_states[i] in action_rate:
            dot_str += "s0 -> s" + str(i+1) + "  [label=\"action:" + str(action_rate[possible_states[i]]) + "\"];\n"
        else:
            dot_str += "s0 -> s" + str(i+1) + "  [label=\"action:0.0\"];\n"
    dot_str += "s0 -> s31  [label=\"end:1\"];\n"

    for j in range(len(possible_states)):
        index = j
        state1 = possible_states[j]
        dot_str += "s" + str(index+1) + " -> s0  [label=\"init:1.0\"];\n"
        for k in range(len(possible_states)):
            index2 = k
            state2 = possible_states[k]
            if transition_matrix[state1][state2] != 0:
                dot_str += "s" + str(index+1) + " -> s" + str(index2+1) + "  [label=\"" + "action" + ":" + str(transition_matrix[state1][state2]) + "\"];\n"
        dot_str += 's' + str(index+1) + ' -> s31' + ' [label="end:1.0"];\n'
    
    dot_str += 's31 -> s31 [label="init:1.0"];\n'
    dot_str += 's31 -> s31  [label="action:1.0"];\n'
    dot_str += 's31 -> s31  [label="end:1.0"];\n'

    dot_str += "__start0 [label=\"\", shape=none];\n"
    dot_str += "__start0 -> s0  [label=\"\"];\n"
    dot_str += "}"

    return dot_str

dot_str = convert_to_dot(transition_matrix)
print(dot_str)

# write the dot string to a file
with open(f"learned_mdp_abcd_flow_{data_part}.dot", "w") as f:
    f.write(dot_str)


digraph learned_mdp {
s0 [label="init"];
s1 [label="pull-up-account"];
s2 [label="enter-details"];
s3 [label="verify-identity"];
s4 [label="make-password"];
s5 [label="search-timing"];
s6 [label="search-policy"];
s7 [label="validate-purchase"];
s8 [label="search-faq"];
s9 [label="membership"];
s10 [label="search-boots"];
s11 [label="try-again"];
s12 [label="ask-the-oracle"];
s13 [label="update-order"];
s14 [label="promo-code"];
s15 [label="update-account"];
s16 [label="search-membership"];
s17 [label="make-purchase"];
s18 [label="offer-refund"];
s19 [label="notify-team"];
s20 [label="record-reason"];
s21 [label="search-jeans"];
s22 [label="shipping-status"];
s23 [label="search-shirt"];
s24 [label="instructions"];
s25 [label="search-jacket"];
s26 [label="log-out-in"];
s27 [label="select-faq"];
s28 [label="subscription-status"];
s29 [label="send-link"];
s30 [label="search-pricing"];
s31 [label="end"];
s0 -> s0  [label="init:1.0"];
s0 -> s1  [label="action:0.10665137614678899"];
s0 -> s2 

In [28]:
from aalpy.utils import load_automaton_from_file, save_automaton_to_file, visualize_automaton, generate_random_dfa
from aalpy.automata import Dfa
from aalpy.SULs import AutomatonSUL
from aalpy.oracles import RandomWalkEqOracle
from aalpy.learning_algs import run_Lstar, run_KV

# load an automaton
automaton = load_automaton_from_file(f"learned_mdp_abcd_flow_{data_part}.dot", automaton_type='mdp')

print(automaton)

# visualize the automaton
visualize_automaton(automaton, path=f"learned_mdp_abcd_flow_{data_part}.png")

digraph learnedModel {
s0 [label="init"];
s1 [label="pull-up-account"];
s2 [label="enter-details"];
s3 [label="verify-identity"];
s4 [label="make-password"];
s5 [label="search-timing"];
s6 [label="search-policy"];
s7 [label="validate-purchase"];
s8 [label="search-faq"];
s9 [label="membership"];
s10 [label="search-boots"];
s11 [label="try-again"];
s12 [label="ask-the-oracle"];
s13 [label="update-order"];
s14 [label="promo-code"];
s15 [label="update-account"];
s16 [label="search-membership"];
s17 [label="make-purchase"];
s18 [label="offer-refund"];
s19 [label="notify-team"];
s20 [label="record-reason"];
s21 [label="search-jeans"];
s22 [label="shipping-status"];
s23 [label="search-shirt"];
s24 [label="instructions"];
s25 [label="search-jacket"];
s26 [label="log-out-in"];
s27 [label="select-faq"];
s28 [label="subscription-status"];
s29 [label="send-link"];
s30 [label="search-pricing"];
s31 [label="end"];
s0 -> s0  [label="init:1.0"];
s0 -> s1  [label="action:0.11"];
s0 -> s2  [label="actio

Model saved to learned_mdp_abcd_flow_all.pdf.
